# Setting-up environment

In [1]:
!cp -r ../input/updated-code-3-7/* /kaggle/working/

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 15.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 44.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 kB 31.6 MB/s eta 0:00:00
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=464219 sha256=a8e75116115a32294506f826647dd38b30aea382328c49e4c2609cbb72693dc2
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
  Created wheel for jupyter-contrib-nbextensions: filename=jupyter_contrib_nbextensions-0.7.0-py2.py3-none-any.whl size=23428781 sha256=6e8b83ec1cb42b448b1238142dc58759dcacfff1e9731f7942a34268f98b0811
  Stored in directory: /root/.cache/pip/wheels/ea/cc/7d

In [3]:
!pip install lightfm tqdm optuna ipykernel matplotlib implicit

  Obtaining dependency information for implicit from https://files.pythonhosted.org/packages/cd/cc/deac70cae8cc32c9885d0cd73bc66e1b3cbea36ae7080b8c83995eaf5322/implicit-0.7.2-cp310-cp310-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 19.6 MB/s eta 0:00:0000:0100:01


In [4]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

# Imports

In [5]:
import time
import optuna
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import vstack, csr_matrix
from scipy.stats import loguniform
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k
import matplotlib.pyplot as plt
from multiprocessing import cpu_count

In [6]:
from Recommenders.BaseRecommender import BaseRecommender
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.NonPersonalizedRecommender import TopPop

#---- CF
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

#---- Matrix Factorization
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#---- CF w/ ML
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from implicit.als import AlternatingLeastSquares

#---- Others
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Recommender_utils import check_matrix

In [7]:
seed = 69
np.random.seed(seed)

# Data Import and Preprocessing

In [8]:
# Import training data
URM_path = "../input/data-books/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                header=0,
                                dtype={0:int, 1:int, 2:int},
                                engine='python')

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]

In [9]:
# Import target users
target_path = "../input/data-books/data_target_users_test.csv"
target_dataframe= pd.read_csv(filepath_or_buffer=target_path,
                                header=0,
                                dtype={0:int},
                                engine='python')
target_dataframe.columns = ["user_id"]
target_dataframe

,user_id
0,1
1,2
2,3
3,4
4,5
...,...
10877,13020
10878,13021
10879,13022
10880,13023


In [10]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings

In [11]:
# Call preprocess data function
ratings = preprocess_data(URM_all_dataframe)

12638 1 13024
22222 1 22347


## Conversion to Sparse Matrix

In [12]:
URM = sps.coo_matrix((ratings.interaction.values, (ratings.mapped_user_id.values, ratings.mapped_item_id.values)))

## Data Splits

In [13]:
#urm_train, urm_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
#evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2127 (16.8%) Users that have less than 1 test interactions


# **Custom Models**

In [14]:
class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha=0.5):
        self.alpha = alpha

    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [15]:
class LightFMCFRecommender(BaseRecommender):
    """LightFMCFRecommender"""

    RECOMMENDER_NAME = "LightFMCFRecommender"

    def __init__(self, URM_train):
        super(LightFMCFRecommender, self).__init__(URM_train)
    
    
    def fit(self, epochs = 300, user_alpha=1e-6, item_alpha = 1e-6, n_factors = 10, n_threads = 4, max_sampled=3, loss='warp', learning_schedule='adagrad'):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss=loss,
                                     user_alpha=user_alpha,
                                     item_alpha=item_alpha,
                                     no_components=n_factors,
                                     max_sampled=max_sampled,
                                     learning_schedule=learning_schedule)

        self.lightFM_model = self.lightFM_model.fit(self.URM_train, 
                                       epochs=epochs,
                                       num_threads=n_threads,
                                       verbose=True)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute)

        return item_scores

In [16]:
class GeneralizedLinearHybridRecommender(BaseRecommender):
    """
    This recommender merges N recommendes by weighting their ratings
    """

    RECOMMENDER_NAME = "GeneralizedLinearHybridRecommender"

    def __init__(self, URM_train, recommenders: list, verbose=True):
        self.RECOMMENDER_NAME = ''
        for recommender in recommenders:
            self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + recommender.RECOMMENDER_NAME[:-11]
        self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + 'HybridRecommender'

        super(GeneralizedLinearHybridRecommender, self).__init__(URM_train, verbose=verbose)

        self.recommenders = recommenders

    def fit(self, alphas=None):
        self.alphas = alphas

    def save_model(self, folder_path, file_name=None):
        pass

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        result = self.alphas[0]*self.recommenders[0]._compute_item_score(user_id_array,items_to_compute)
        for index in range(1,len(self.alphas)):
            result = result + self.alphas[index]*self.recommenders[index]._compute_item_score(user_id_array,items_to_compute)
        return result

In [17]:
class FastIALSRecommender(BaseMatrixFactorizationRecommender):
    RECOMMENDER_NAME = "FastIALSRecommender"

    AVAILABLE_CONFIDENCE_SCALING = ["linear", "log"]
    
    def __init__(self, URM_train, verbose=True):
        super().__init__(URM_train, verbose=verbose)
        
    def fit(self,
            factors=20,
            regularization=1e-3,
            iterations=100,
            calculate_training_loss=False,
            num_threads=0,
            confidence_scaling='linear',
            alpha=1.0,
            epsilon=0,
            #---- Do not change
            use_native=True,
            use_cg=True,
            use_gpu=True):
        if confidence_scaling not in self.AVAILABLE_CONFIDENCE_SCALING:
           raise ValueError("Value for 'confidence_scaling' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_CONFIDENCE_SCALING, confidence_scaling))

        self.alpha = alpha
        self.epsilon = epsilon
        self.num_factors = factors
        self.reg = regularization
        
        self.USER_factors = self._init_factors(self.n_users, False)  # don't need values, will compute them
        self.ITEM_factors = self._init_factors(self.n_items)
        
        self.recommender = AlternatingLeastSquares(factors=factors, regularization=regularization,
                                                        use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
                                                        iterations=iterations,
                                                        calculate_training_loss=calculate_training_loss,
                                                        num_threads=num_threads)
        
        self._build_confidence_matrix(confidence_scaling)
        self.recommender.fit(self.C, show_progress=self.verbose)
        
        self.USER_factors = self.recommender.user_factors.to_numpy()
        self.ITEM_factors = self.recommender.item_factors.to_numpy()
        
    
    def _linear_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha*C.data

        return C

    def _log_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha * np.log(1.0 + C.data / self.epsilon)

        return C
    
    def _build_confidence_matrix(self, confidence_scaling):

        if confidence_scaling == 'linear':
            self.C = self._linear_scaling_confidence()
        else:
            self.C = self._log_scaling_confidence()

        self.C_csc= check_matrix(self.C.copy(), format="csc", dtype = np.float32)
    
    def _init_factors(self, num_factors, assign_values=True):

        if assign_values:
            return self.num_factors**-0.5*np.random.random_sample((num_factors, self.num_factors))

        else:
            return np.empty((num_factors, self.num_factors))

# **Best Model Params**

In [18]:
ItemKNN_params = {
    'topK': 6,
    'shrink': 15,
    'similarity': 'jaccard',
    'normalize': False
}

In [19]:
RP3beta_params = {
    'alpha': 0.307953246083667, 
    'beta': 0.3073797221110665, 
    'topK': 59, 
    'normalize_similarity': True
}

In [20]:
alpha_itemknn_rp3beta=0.7381515719042592

In [21]:
UserKNN_params = {
    'topK': 470,
    'shrink': 0,
    'similarity': 'cosine',
    'normalize': True
}

In [22]:
LightFM_params = {
                  'n_factors': 482,
                  'max_sampled': 5,
                  'user_alpha': 0.00023989649900734266,
                  'item_alpha': 9.740651135253414e-05
                 }

In [23]:
SLIM_params = {
    'l1_ratio': 0.013752256221164005,
    'alpha': 0.0031943927190071775,
    'topK': 622
}

In [24]:
NMF_params = {
    'l1_ratio': 0.005734775635120469,
    'num_factors': 134,
    'beta_loss': 'frobenius',
    'init_type': 'nndsvda',
    'solver': 'multiplicative_update'
}

In [25]:
IALS_params = {
    'confidence_scaling': 'log',
    'epsilon': 0.11624415533664904,
    'factors': 116,
    'regularization': 0.005454427904241962,
    'alpha': 1.7221339971074425
}

# Tuning UserKNN

In [26]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        # Create a DataFrame from the current trial's results
        trial_df = pd.DataFrame([hyperparam_dict])

        # Use concat instead of append
        self.results_df = pd.concat([self.results_df, trial_df], ignore_index=True)

In [27]:
def objective_function_UserKNN(optuna_trial):
    topK = optuna_trial.suggest_int("topK", 5, 500)
    shrink = optuna_trial.suggest_int("shrink", 0, 2500)
    similarity = optuna_trial.suggest_categorical("similarity", ["cosine","pearson","jaccard","tanimoto","adjusted","euclidean"])
    normalize = optuna_trial.suggest_categorical("normalize", [True, False])
    feature_weighting = optuna_trial.suggest_categorical("feature_weighting", ["BM25", "TF-IDF", "none"])

    recommender_instance = UserKNNCFRecommender(urm_train)
    recommender_instance.fit(
                             topK = topK,
                             shrink = shrink,
                             similarity = similarity,
                             normalize = normalize,
                             feature_weighting = feature_weighting
                            )

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [ ]:
optuna_study_UserKNN = optuna.create_study(direction="maximize")

save_results_UserKNN = SaveResults()

optuna_study_UserKNN.optimize(objective_function_UserKNN,
                      callbacks=[save_results_UserKNN],
                      n_trials = 2000)

[I 2023-12-07 08:55:22,940] A new study created in memory with name: no-name-c0229118-5d86-4a55-9098-0b4c96c01387


UserKNNCFRecommender: URM Detected 240 ( 1.9%) users with no interactions.
UserKNNCFRecommender: URM Detected 123 ( 0.6%) items with no interactions.
Similarity column 12638 (100.0%), 812.75 column/sec. Elapsed time 15.55 sec
EvaluatorHoldout: Processed 10511 (100.0%) in 9.05 sec. Users per second: 1162


[I 2023-12-07 08:55:47,624] Trial 0 finished with value: 0.0006449101696937291 and parameters: {'topK': 35, 'shrink': 372, 'similarity': 'euclidean', 'normalize': False, 'feature_weighting': 'BM25'}. Best is trial 0 with value: 0.0006449101696937291.


UserKNNCFRecommender: URM Detected 240 ( 1.9%) users with no interactions.
UserKNNCFRecommender: URM Detected 123 ( 0.6%) items with no interactions.
Similarity column 12638 (100.0%), 4297.61 column/sec. Elapsed time 2.94 sec
EvaluatorHoldout: Processed 10511 (100.0%) in 13.41 sec. Users per second: 784


[I 2023-12-07 08:56:04,481] Trial 1 finished with value: 0.02584208455842943 and parameters: {'topK': 383, 'shrink': 91, 'similarity': 'adjusted', 'normalize': False, 'feature_weighting': 'TF-IDF'}. Best is trial 1 with value: 0.02584208455842943.


...

UserKNNCFRecommender: URM Detected 240 ( 1.9%) users with no interactions.
UserKNNCFRecommender: URM Detected 123 ( 0.6%) items with no interactions.
Similarity column 12638 (100.0%), 4184.83 column/sec. Elapsed time 3.02 sec
EvaluatorHoldout: Processed 10511 (100.0%) in 13.77 sec. Users per second: 763


[I 2023-12-07 10:20:16,139] Trial 287 finished with value: 0.02649078893917647 and parameters: {'topK': 380, 'shrink': 106, 'similarity': 'adjusted', 'normalize': True, 'feature_weighting': 'TF-IDF'}. Best is trial 102 with value: 0.03490503901430594.


UserKNNCFRecommender: URM Detected 240 ( 1.9%) users with no interactions.
UserKNNCFRecommender: URM Detected 123 ( 0.6%) items with no interactions.
Similarity column 12638 (100.0%), 4490.82 column/sec. Elapsed time 2.81 sec
EvaluatorHoldout: Processed 10511 (100.0%) in 13.89 sec. Users per second: 757


[I 2023-12-07 10:20:33,148] Trial 288 finished with value: 0.03461752842449303 and parameters: {'topK': 327, 'shrink': 213, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'}. Best is trial 102 with value: 0.03490503901430594.


UserKNNCFRecommender: URM Detected 240 ( 1.9%) users with no interactions.
UserKNNCFRecommender: URM Detected 123 ( 0.6%) items with no interactions.
Similarity column 12638 (100.0%), 4483.51 column/sec. Elapsed time 2.82 sec


In [ ]:
pruned_trials = [t for t in optuna_study_UserKNN.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study_UserKNN.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study_UserKNN.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study_UserKNN.best_trial.value)

print("Best params:")
print(optuna_study_UserKNN.best_trial.params)